# ButtPlug Wars

## Setup

In [1]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import { smock } from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'
import * as contracts from './utils/contracts'

import {getMainnetSdk} from '@dethcrypto/eth-sdk-client';

In [2]:
const FIVE_OUT_OF_NINE = '0xB543F9043b387cE5B3d1F0d916E42D8eA2eBA2E0';
const WETH_9 = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2';
const KEEP3R = '0xeb02addCfD8B773A5FFA6B9d1FE99c566f8c44CC';
const KP3R_LP = '0x3f6740b5898c5D3650ec6eAce9a649Ac791e44D7';
const SUDOSWAP_FACTORY = '0xb16c1342E617A5B6E4b631EB114483FDB289c0A4';
const SUDOSWAP_XYK_CURVE = '0x7942E264e21C5e6CbBA45fe50785a15D3BEb1DA0';

const FIVEOUTOFNINE_WHALE = '0xC5233C3b46C83ADEE1039D340094173f0f7c1EcF'
const KEEPER_ADDRESS = '0x9429cd74a3984396f3117d51cde46ea8e0e21487'

In [3]:
// FORK
let blockNumber = 16000000

await evm.reset({
    jsonRpcUrl: process.env['MAINNET_RPC'],
    blockNumber
})

const provider = ethers.provider

In [4]:
// Players
const signer = await ethers.getSigner()
const whale = await wallet.impersonate(FIVEOUTOFNINE_WHALE)
const keeper = await wallet.impersonate(KEEPER_ADDRESS)

In [5]:
// Deployment
const args = [FIVE_OUT_OF_NINE, WETH_9, KEEP3R, KP3R_LP, SUDOSWAP_FACTORY, SUDOSWAP_XYK_CURVE]
const game = await contracts.deploy(signer, './out/ButtPlugWars.sol/ButtPlugWars.json', args)

In [6]:
const { chess, sudoPool } = getMainnetSdk(signer)

In [7]:
// JSON.parse(atob((await chess._tokenURI(100)).substring(29,1e6)))

## Playground

### Event start

In [8]:
await evm.advanceTimeAndBlock(86400*10)
const tx = await game.startEvent();

### Player Badge minting

In [9]:
// pre-genesis tokens
await chess.connect(whale).transferFrom(whale._address, signer.address, 133)
await chess.connect(whale).transferFrom(whale._address, signer.address, 134)
await chess.connect(whale).transferFrom(whale._address, signer.address, 135)
await chess.connect(whale).transferFrom(whale._address, signer.address, 136)
await chess.connect(whale).transferFrom(whale._address, signer.address, 137)
await chess.connect(whale).transferFrom(whale._address, signer.address, 138)
await chess.setApprovalForAll(game.address, true)

await game.mintPlayerBadge(133, 0, {value: bn.toUnit(1)})
await game.mintPlayerBadge(134, 0, {value: bn.toUnit(0.5)})
await game.mintPlayerBadge(135, 0, {value: bn.toUnit(0.25)})
await game.mintPlayerBadge(136, 0, {value: bn.toUnit(0.10)})
await game.mintPlayerBadge(137, 0, {value: bn.toUnit(0.10)})
const tx = await game.mintPlayerBadge(138, 1, {value: bn.toUnit(1)})

In [10]:
await game['getBadgeId(uint256)'](5)

BigNumber { _hex: '0x05', _isBigNumber: true }


In [11]:
await game['getBadgeId(uint256)'](6)

BigNumber { _hex: '0x0100000006', _isBigNumber: true }


### ButtPlug Badge minting

In [12]:
const buttPlug = await contracts.deploy(signer,'./out/Common.sol/ButtPlugForTest.json',[])

const tx = await game.mintButtPlugBadge(buttPlug.address)

### Game start

In [13]:
await evm.advanceTimeAndBlock(14 * 86400)
const tx = await game.pushLiquidity()

### Voting

In [14]:
const tx = await game['voteButtPlug(address,uint256)'](buttPlug.address, 1);
// const tx = await game['voteButtPlug(address,uint256[])'](buttPlug.address, [1, 2]);

await contracts.logTx(tx)

gasUsed 101375
101375


## Gameplay

In [15]:
const officialSudoPool = sudoPool.attach(await game.SUDOSWAP_POOL())

In [16]:
await chess.board()

BigNumber {
  _hex: '0x03256230011111100000000000000000099999900bcdecb000000001',
  _isBigNumber: true
}


### E2E environment

In [17]:
await chess.connect(whale).transferFrom(whale._address, keeper._address, 175)
'keepers need a 5/9'

keepers need a 5/9


In [18]:
await buttPlug.setDepth(7)
await evm.advanceTimeAndBlock(5*86400)
const tx = await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

59295000000000000


In [19]:
const tx = await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

0


### Post Genesis badges

In [20]:
const receipt = await tx.wait()
const postGenesisToken = receipt.events[3].topics[3]

await game.mintPlayerBadge(postGenesisToken, 1, {value: bn.toUnit(0.1)})

{
  hash: '0x288ce6c1e148c10cd03df2a9781923f807a018e5178dcbff686ced6af8864458',
  type: 2,
  accessList: [],
  blockHash: '0x2f817ecb8ce47014b96175ee4d413b187ee596bfbd6447852e078b2b898ffe3c',
  blockNumber: 16000027,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x54a4b809', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x6daea612', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad5d8', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x016345785d8a0000', _isBigNumber: true },
  nonce: 364,
  data: '0xfbaedbfa00000000000000000000000000000000000000000000000000000000000000c20000000000000000000000000000000000000000000000000000000000000001',
  r: '0x9a473bae36a1fb02d146d071ce1ce4f9072eb772996e2429b0e4d34a3d4ecc78',
  s: '0x1331bae56aea84423fb4f014dd8e9149af4c

In [21]:
await game['getBadgeId(uint256)'](7)

BigNumber { _hex: '0x0100000007', _isBigNumber: true }


### Mocked environment

In [22]:
const iButtPlug = await contracts.getContractAbi('./out/IGame.sol/IButtPlug.json')
const fakeButtPlug = await smock.fake(iButtPlug)

#### Mock vs FiveOutOfNine

In [23]:
const tx = await game['voteButtPlug(address,uint256[])'](fakeButtPlug.address, [1, 2, 3, 4, 5]);

In [24]:
// Game #6: The Royal Entrance

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((10 << 6) | 25)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((13 << 6) | 30)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((20 << 6) | 28)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((12 << 6) | 28)
await game.connect(keeper).executeMove()

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((28 << 6) | 42)
await game.connect(keeper).executeMove()

{
  hash: '0x7d134722e962a5bb88e4fdd5e3e001b29a0d84b44bfb49ac1b1284da4bea6424',
  type: 2,
  accessList: [],
  blockHash: '0xe01ca48411720d90a32210c11b950ffb8ccedc2cc9ee2521b64c194b59a651d2',
  blockNumber: 16000038,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x9429cd74A3984396f3117d51cde46ea8e0e21487',
  gasPrice: BigNumber { _hex: '0x41999ea9', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x47987352', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 507,
  data: '0x30e6d8b2',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


#### Fake Wars

In [25]:
const chessAbi = require('./eth-sdk/abis/mainnet/chess.json')
const keep3rAbi = await contracts.getContractAbi('./out/IKeep3r.sol/IKeep3r.json')
const fakeChess = await smock.fake(chessAbi, {address: chess.address})
fakeChess.balanceOf.whenCalledWith(keeper._address).returns(9)
fakeChess.balanceOf.whenCalledWith(officialSudoPool.address).returns(100)

fakeChess.transferFrom.returns(true)

const fakeKeep3r = await smock.fake(keep3rAbi, {address: KEEP3R})
fakeKeep3r.isKeeper.returns(true)
fakeKeep3r.worked.returns

[Function: bound returns]


In [26]:
const NEW_BOARD = '0x03256230011111100000000000000000099999900bcdecb000000001'
const WHITE_CAP = '0x03256230011111100000000000000000099909900bcdecb000000001'
const BLACK_CAP = '0x03256230011011100000000000000000099999900bcdecb000000001'
const BOTH_CAPS = '0x03256230011011100000000000000000099909900bcdecb000000001'

In [27]:
fakeChess.board.reset();
fakeChess.board.returnsAtCall(0, NEW_BOARD)
fakeChess.board.returnsAtCall(1, BLACK_CAP)
await evm.advanceTimeAndBlock(5*86400)
await game.connect(keeper).executeMove()

{
  hash: '0xbfa2a39e025be888c0934d2ab214a9665f7c8cbb1f445677e86ed0d27e6b27b9',
  type: 2,
  accessList: [],
  blockHash: '0x7d41bb96096883997ff27c117aa27857056326930ff912d7563c99738ef89386',
  blockNumber: 16000040,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x9429cd74A3984396f3117d51cde46ea8e0e21487',
  gasPrice: BigNumber { _hex: '0x403441cc', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x44cdb998', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 508,
  data: '0x30e6d8b2',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [28]:
const ZERO = '0x0000000000000000000000000000000000000000'
await wallet.fund(fakeChess.address, bn.toUnit(1))

await game.connect(fakeChess.wallet).onERC721Received(ZERO, ZERO, 1, ZERO)
await officialSudoPool.connect(fakeChess.wallet).onERC721Received(ZERO, ZERO, 1, ZERO)
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

177885000000000000


## Token URIs

In [29]:
fakeChess.board.returns('0x0335423001101110000006009000c000099b09e00bc9ecb000000001')

const scoreboard = await game.tokenURI(0)

In [30]:
JSON.parse(atob(scoreboard.substring(29,1e6)))

{
  name: 'ButtPlugWars Scoreboard',
  description: 'Scoreboard NFT with information about the game state',
  image_data: '',
  attributes: [
    { trait_type: 'game-score', value: '0(0) - 0(0)' },
    { trait_type: 'weight', value: 4155 }
  ]
}


In [31]:
JSON.parse(atob((await game.tokenURI(1)).substring(29,1e6)))

{
  name: 'Player',
  description: 'Player Badge with FiveOutOfNine#133',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ZERO' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: 0 },
    { trait_type: 'vote', value: '0x427719c8' },
    { trait_type: 'bonded_token', value: '133' }
  ]
}


In [32]:
fakeButtPlug.owner.returns(fakeButtPlug.address)
await game.mintButtPlugBadge(fakeButtPlug.address)

{
  hash: '0x751744f7685f1a06ec9645e243195107b631cade80b44fcdd2b30bb094deeb82',
  type: 2,
  accessList: [],
  blockHash: '0x8f48f847f30481bbaf6ac2293eedc0dc0758ecaf91afe551e24ae2ea4b6b52c9',
  blockNumber: 16000044,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3e4ef25e', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x41031abc', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad9d8', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 367,
  data: '0x067e5741000000000000000000000000427719c8fb55f3f5dba2bdc22fc183879d50d4f5',
  r: '0x57dc4ae12ea9005d70fe0d9f8bccb5651e3fdb5224dfab74acdab582f06632a3',
  s: '0x027f87d86d24e50b02c0f1c745fcb7a55123ce75cba8abcf3aabb4f69527a5d5',
  v: 1,
  creates: null,
  chainId: 31337,
  wa

In [33]:
const fakeChess = await smock.fake(chessAbi, {address: '0x2ea2736Bfc0146ad20449eaa43245692E77fd2bc'})
fakeChess.board.returns(WHITE_CAP)

In [34]:
fakeButtPlug.readMove.returns(2731)

In [35]:
const buttPlugBadge = await game['getBadgeId(address)'](buttPlug.address)
const buttPlugURI = await game.tokenURI(buttPlugBadge)

In [37]:
JSON.parse(atob(buttPlugURI.substring(29,1e6)))

{
  name: 'ButtPlug',
  description: 'ButtPlug Badge for contract at 0xf66cfdf074d2ffd6a4037be3a669ed04380aef2b',
  image_data: '',
  attributes: [
    { trait_type: 'score', value: 0 },
    { trait_type: 'simulated_move', value: 'c1 queen to d2' },
    { trait_type: 'simulated_gas', value: 4200395 },
    { trait_type: 'is_legal_move', value: true }
  ]
}


In [40]:
fakeButtPlug.readMove.returns(635)
JSON.parse(atob(buttPlugURI.substring(29,1e6)))

{
  name: 'ButtPlug',
  description: 'ButtPlug Badge for contract at 0xf66cfdf074d2ffd6a4037be3a669ed04380aef2b',
  image_data: '',
  attributes: [
    { trait_type: 'score', value: 0 },
    { trait_type: 'simulated_move', value: 'c1 queen to d2' },
    { trait_type: 'simulated_gas', value: 4200395 },
    { trait_type: 'is_legal_move', value: true }
  ]
}


unknown msg_type: comm_open
unknown msg_type: comm_msg
unknown msg_type: comm_open
unknown msg_type: comm_msg


## Prize claiming

In [ ]:
const tx = await game.unbondLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game['claimPrize(uint256)'](5)
// const tx = await game['claimPrize(uint256[])']([0,1,2,3,4])

await contracts.logTx(tx)

### Prize ceremony

In [ ]:
await evm.advanceTimeAndBlock(14*86400)
const tx = await game.withdrawLiquidity()

await contracts.logTx(tx)

In [ ]:
const tx = await game.withdrawPrize()

await contracts.logTx(tx)